## 1 Classifying Extrema 
function to calculate df ddf

In [13]:
import numpy as np
import sympy as sp
def getd(f_str, val):
    x = sp.symbols('x')
    formula = sp.sympify(f_str)
    df = sp.diff(formula, x)
    ddf = sp.diff(df, x)
    d = df.evalf(subs={x:val})
    dd = ddf.evalf(subs={x:val})
    return df, ddf, d, dd
f = "-1*(x**3)/3 + 3*(x**2)/2 + 3*x + 12"
print(getd(f, -0.791))

print(getd(f, 3.791))


(-x**2 + 3*x + 3, 3 - 2*x, 0.00131899999999983, 4.58200000000000)
(-x**2 + 3*x + 3, 3 - 2*x, 0.00131900000000034, -4.58200000000000)


## 2.Determine the length of the interval

In [15]:
'''
tau: 系数
a0: left bound
b0: right bound
f: function
'''
import numpy as np
a = 17.09
b = 9.79
c = 0.6317
d = 0.9324
e = 0.4565

def f(x):
    return a*x**4 + b*x**3 + c*x**2 + d*x + e

def df(x):
    return 4*a*x**3 + 3*b*x**2 + 2*c*x + d

def d2f(x):
    return 3*4*a*x**2 + 2*3*b*x + 2*c
tau = (np.sqrt(5)-1)/2
a0 = -7 #-2
b0 = 12 #1


h_k = b0 - a0  
x1 = a0 + (1-tau) * h_k
x2 = a0 + tau * h_k
#print(x1,x2)
f1 = f(x1)
f2 = f(x2)

errors = [np.abs(h_k)]
count = 0

while (count < 30 and np.abs(h_k) > 1e-6):
 
    if  f1>f2:
        a0 = x1
        x1 = x2
        f1 = f2
        h_k = b0-a0
        x2 = a0 + tau * h_k
        f2 = f(x2)
        count = count + 1
        if count == 4:
            print(h_k)
    else:
        b0 = x2
        x2 = x1
        f2 = f1
        h_k = b0-a0
        x1 = a0 + (1-tau) * h_k
        f1 = f(x1)   
        count = count + 1
        if count == 4:
            print(h_k)
        
    errors.append(np.abs(h_k))  
    
    #print("%10g \t %10g \t %12g %12g" % (a0, b0, errors[-1], errors[-1]/errors[-2]))
    


2.7720626412559946


## 16.2. Newton approximation

In [1]:
import numpy as np
x0 = 2
def f(x):
    return 17*x**3 + 21*x**2 + 4*x + 2
def df(x):
    return 51*x**2 + 42*x + 4
def df2(x):
    return 102*x + 42
c = f(x0)
b = df(x0)
a = 0.5*df2(x0)
print(a," ",b," ",c)

123.0   292   230


## 16.4. Carrying out Newton steps

In [13]:
#-e^(-x^2)
import numpy as np
def df(x):
    return 2*x*np.exp(-x**2)
def d2f(x):
    return 2*np.exp(-x**2)*(1-2*x**2)
x = 0.4 
for i in range(1):
    dfx = df(x)
    d2fx = d2f(x)
    xnew = x - dfx / d2fx
    if np.abs(xnew-x) < 1e-8:
        break
    x = xnew 
print(x)

-0.18823529411764717


In [12]:
#cosx 
import numpy as np
def df(x):
    return -1*np.sin(x)
def d2f(x):
    return -1*np.cos(x)
x = 0.4 
for i in range(1):
    dfx = df(x)
    d2fx = d2f(x)
    xnew = x - dfx / d2fx
    if np.abs(xnew-x) < 1e-8:
        break
    x = xnew 
print(x)

-0.022793218738161758


## 16.10. Newton on 2D Quadratic Function
find iteration

In [24]:
import numpy as np
import numpy.linalg as la
import sympy as sp
def gradient(formula, symbols, values=None):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae or numerical gradient
    if values specified
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
        
    if values == None:
        return gradient

    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at
def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())
def hessian(gradient, symbols, values=None):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    or numerical Hessian if values are specified
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
            
    if values == None:
        return hessian
    
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            hessian_at[i][j] = subs_all(hessian_at[i][j], symbols, [float(v) for v in values])
#             for k in range(len(symbols)):
#                 hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
#             hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at
def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    hes_at = np.array(hessian(grad, symbols, values=x_prev))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at)
def newton_nd_optimization_crude(f_str, s_str, start, tolerance):
    '''
    A crude version of Newton ND Optimization algorithm
    Maybe you can help out implementing an analytic solution
    Returning the numerical solution as well as the number of
    iterations it took
    '''
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    last = curr
    maxi = 100
    for i in range(maxi):
        last = curr
        
        curr = newton_nd_optimization_crude_step(formula, symbols, curr)
        if abs(la.norm(curr - last, 2))< tolerance:
            break
        iterations += 1
    return curr, iterations
# Workspace
'''
form = sp.sympify('16*x**2 - 6*x + 6 + 14*y**2 - x*y')
symbols = sp.symbols('x y')
values = [1, 2]
hes = hessian(gradient(form, symbols), symbols, values)
x_1 = newton_nd_optimization_crude_step(form, symbols, values)
step = np.array(x_1) - np.array(values)
print(hes)
print(x_1)
print(step)
'''
f_str = '25*x**2 - 5*x + 5 + 21*y**2 - x*y'
s_str = 'x y'
start = [1, 2]
tolerance = 10**-9
ans = newton_nd_optimization_crude(f_str, s_str, start, tolerance)
print(ans)

(array([0.10004764, 0.00238209]), 1)


## 16.11 N-Dimension Optimization in Steps
get accmulation

In [41]:
import numpy as np
import numpy.linalg as la
import sympy as sp
def gradient(formula, symbols, values=None):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae or numerical gradient
    if values specified
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
        
    if values == None:
        return gradient

    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at
def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())
def hessian(gradient, symbols, values=None):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    or numerical Hessian if values are specified
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
            
    if values == None:
        return hessian
    
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            hessian_at[i][j] = subs_all(hessian_at[i][j], symbols, [float(v) for v in values])
#             for k in range(len(symbols)):
#                 hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
#             hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at
def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    hes_at = np.array(hessian(grad, symbols, values=x_prev))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at)

form = sp.sympify('8*x**2 + 3*x*y + 8*y**2 + 10*E**(7*x*y) + 7*(sin(y)**2) + 14*cos(x*y)')
symbols = sp.symbols('x y')
values = [0, 1]
grad = gradient(form, symbols, values)
hes = hessian(gradient(form, symbols), symbols, values)
x_1 = newton_nd_optimization_crude_step(form, symbols, values)
step = np.array(x_1) - np.array(values)
print("grad is ", grad)
print("hes is ", hes)
print("x1 is", x_1)
print("s0 is", step)



grad is  [73.0, 22.36508198777977]
hes is  [[492.0, 73.0], [73.0, 10.173944288340007]]
x1 is [-2.75169957 18.54570122]
s0 is [-2.75169957 17.54570122]


In [ ]:
def f2(x):
    a = x[0]
    b = y[0]
    return (x[0]-1)**2 + (x[1]-1)**2

def df2(x):
    return np.array([2*(x[0]-1),2*(x[1]-1) ])

def ddf2(x):
    return np.array([
                     [2,0],
                     [0,2]
                     ])
def steepestDescent(f,df,x0,maxiter,tol,alpha = 0):
    # Line search function
    def f_line(alpha):
        fnew = f(x + alpha*s)
        return fnew
    steps = [x0]   
    x = x0
    fhist = [f(x)]
    # Steepest descent with line search
    for i in range(maxiter):
        # Get the gradient
        s = -df(x)
        # Learning rate:
        if alpha == 0:
            # Perform line search
            alpha_opt = sopt.golden(f_line)
        else:
            alpha_opt = alpha
        # Steepest descent update
        xnew = x + alpha_opt * s
        # Save optimized solution for plotting
        steps.append(xnew)
        fhist.append(f(xnew))
        # Check convergence
        if ( np.abs(fhist[-1] - fhist[-2]) < tol ):
            break
        x = xnew
    print('optimal solution is:', x)
    return steps, fhist, i   

x0 = np.array([-1.5, -1])
# Steepest descent
steps, fhist, iterations = steepestDescent(f2,df2,x0,50,1e-4)
print('converged in', iterations, 'iterations')


## C1 Golden Section Search

In [42]:
import numpy as np
import matplotlib.pyplot as plt
def f(x):
    return (x - 3.2)**2
a = -1
b = 8
brackets = []
gs = (np.sqrt(5) - 1) / 2
m1 = a + (1 - gs) * (b - a)
m2 = a + gs * (b - a)
epsilon = 1e-5
# Begin your modifications below here
f1 = f(m1)
f2 = f(m2)
h = b-a
while h >= epsilon:
    if f1 > f2:
        a = m1
        m1 = m2
        f1 = f2
        h *= gs
        m2 = a + gs * h
        f2 = f(m2)
        brackets.append([a, m1, m2, b])
    else:
        b = m2
        m2 = m1
        f2 = f1
        h *= gs
        m1 = a + (1 - gs) * h
        f1 = f(m1)
        brackets.append([a, m1, m2, b])
    
print(a)
print(b)

3.199994811011908
3.2000026381129847


## C3 Using Newton's Method for 1D Optimization

In [ ]:
import numpy as np

# complete the function below
def dfunc(x):
    # Add your code here
    return np.exp(-1 * x**2)*(2*x**2 + 2*x*np.sin(x)-1-np.cos(x))
    
# complete the function below
def d2func(x):
    return np.exp(-x**2)*(-4*x**3 - 4*np.sin(x)*x**2 +6*x + 4*x *np.cos(x)+3*np.sin(x))
    
newton_guesses = [x0]
x = x0
for i in range(100):
    if dfunc(x0) < tol:
        break
        
    dfx = dfunc(x)
    d2fx = d2func(x)
    xnew = x - dfx / d2fx
    newton_guesses.append(xnew)
    if abs(dfunc(xnew)) <= tol:
        break
    x = xnew 
newton_guesses = np.array(newton_guesses)